## Metro cuadrado scraping

In [1]:
#Importando librerias
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import numpy as np
import pandas as pd

In [43]:
#Accediendo al driver
driver_path = '../chromedriver_mac64/chromedriver'
chrome_options = Options()
chrome_options.add_argument("--window-size=1920x1080")
driver = webdriver.Chrome(service = Service(driver_path))
url = 'https://www.metrocuadrado.com/apartamentos/arriendo/bogota/'

In [184]:
#Abrir página principal;
driver.get(url)

#### Filtros

In [45]:
#Identificando XPATHS para simular eventos de clicks y scroll que requiero para utilizar los filtros.

In [46]:
boton_filtro_habitaciones_1 = '//input[@id="room-list-1"]/following-sibling::span[@class="custom-control-description"]'

In [47]:
boton_filtro_habitaciones_2 = '//input[@id="room-list-2"]/following-sibling::span[@class="custom-control-description"]'

In [48]:
scroll_element = '//div[@class="save-filter-search mb-2"]'


In [49]:
boton_estrato = '//div[@class="m2-select__value-container m2-select__value-container--is-multi css-1hwfws3"]'

In [52]:
boton_filtro_estrato_4 = '//div[@id="react-select-9-option-3"]'

In [53]:
boton_filtro_estrato_5 = '//div[@id="react-select-9-option-4"]'

In [54]:
scroll_element = '//div[@id="react-select-9-option-7"]'

In [55]:
boton_filtro_estrato_6 = '//div[@id="react-select-9-option-5"]'

#### Página de inicio

In [57]:
apartamentos_web_elements = driver.find_elements(By.XPATH, '//li[@class="sc-gPEVay dibcyk"]/div[@class="card-header"]/a')

In [58]:
#Obteniendo url de cada uno de los apartamentos
apartamentos_urls = []
for i in apartamentos_web_elements:
    href = i.get_attribute('href')
    apartamentos_urls.append(href)

In [59]:
#Abrir nueva pestaña y posicionar driver en la misma.
driver.execute_script("window.open('');")
driver.switch_to.window(driver.window_handles[1])

In [60]:
#Abrir url
driver.get(apartamentos_urls[0])

#### Página de cada apartamento

In [82]:
datos_resaltados_we = driver.find_elements(By.XPATH, '//li[@class="Li-sc-1knn373-0 ctJpwh"]/div[contains(@class, "py-2")]//h2')[1:]

In [62]:
#Obtener valores de habitaciones, baños y estrato. Luego guardarlos en una lista de diccionarios.
datos_resaltados_limpios = []
for i in datos_resaltados_we:
    name = i.text.split('\n')[1]
    value = i.text.split('\n')[0]
    datos_resaltados_limpios.append({'name': name, 'value': value })
datos_resaltados_limpios 




[{'name': 'Habitaciones', 'value': '2'},
 {'name': 'Baños', 'value': '3'},
 {'name': 'Estrato', 'value': '6'}]

In [68]:
#Validar que la información sea consistente. Hay apartamentos que no tienen todos los datos.
for i in datos_resaltados_limpios:
    if 'Habitaciones' in i['name']:
        habitaciones = i['value']
    if 'Baños' in i['name']:
        banos = i['value']
    if 'Estrato' in i['name']:
        estrato = i['value']

In [81]:
datos_principales_we = driver.find_elements(By.XPATH, '//div[@class="Col-sc-14ninbu-0 lfGZKA mb-3 pb-1 col-12 col-lg-3"]')

In [67]:
#Obtener demás valores del apartamento. Luego guardarlos en una lista de diccionarios.
datos_principales_limpios = []
for i in datos_principales_we:
    name = i.find_element(By.XPATH, './h3').text
    value = i.find_element(By.XPATH, './p').text
    datos_principales_limpios.append({'name': name, 'value': value })
datos_principales_limpios

[{'name': 'Código inmueble', 'value': '13036-M4062128'},
 {'name': 'Barrio común', 'value': 'CHICO NORTE III'},
 {'name': 'Antigüedad', 'value': 'Más de 20 años'},
 {'name': 'Área construida', 'value': '98 m²'},
 {'name': 'Área privada', 'value': '98 m²'},
 {'name': 'Valor arriendo', 'value': '$4.000.000'},
 {'name': 'Valor administración', 'value': '$800.000'},
 {'name': 'Parqueaderos', 'value': '1'}]

In [70]:
#Validar que la información sea consistente. Hay apartamentos que no tienen todos los datos.
for i in datos_principales_limpios:
    if 'Código inmueble' in i['name']:
        codigo_inmueble = i['value']
    if 'Barrio común' in i['name']:
        barrio = i['value']
    if 'Antigüedad' in i['name']:
        antiguedad = i['value']
    if 'Área construida' in i['name']:
        area_construida = i['value']
    if 'Área privada' in i['name']:
        area_privada = i['value']
    if 'Valor arriendo' in i['name']:
        valor_arriendo = i['value']
    if 'Valor administración' in i['name']:
        valor_administracion = i['value']
    if 'Parqueaderos' in i['name']:
        parqueadero = i['value']

#### Limpieza de datos

In [71]:
# Dar formato a los datos para extaerlos limpios y facilitar su tratamiento.

In [72]:
barrio.title()

'Chico Norte Iii'

In [73]:
antiguedad

'Más de 20 años'

In [74]:
area_construida.split(' m')[0]

'98'

In [75]:
area_privada.split(' m')[0]

'98'

In [76]:
valor_arriendo.split('$')[1].replace('.','')

'4000000'

In [77]:
valor_administracion.split('$')[1].replace('.','')

'800000'

In [78]:
parqueadero

'1'

In [79]:
sector = driver.find_element(By.XPATH, '//p[@class="P-sc-31hrrf-0 hGwghD card-subtitle"]').text
if 'Sector' in sector:
    sector = sector.split(',')[0].split('Sector ')[1]
else:
    sector = sector.split('Área')[0].strip()
sector

'Chicó'

## Organizando todo en funciones

In [2]:
#Función del driver de Chrome.
def set_driver():
    driver_path = '../chromedriver_mac64/chromedriver'
    chrome_options = Options()
    chrome_options.add_argument("--window-size=1920x1080")
    driver = webdriver.Chrome(service = Service(driver_path))
    driver.get(url)
    WebDriverWait(driver, delay).until(EC.presence_of_element_located((By.XPATH, '//li[@class="sc-gPEVay dibcyk"]/div[@class="card-header"]/a')))

In [3]:
#Función para ejecutar scroll segun el xpath.
def scroll(xpath_route):
    scroll_element = driver.find_element(By.XPATH, xpath_route) 
    actions = ActionChains(driver)
    actions.move_to_element(scroll_element).perform()

In [22]:
#Función para filtrar apartamentos en específico.
def page_filter():
    boton_filtro_habitaciones_1 = driver.find_element(By.XPATH, '//input[@id="room-list-1"]/following-sibling::span[@class="custom-control-description"]').click()
    boton_filtro_habitaciones_2 = driver.find_element(By.XPATH, '//input[@id="room-list-2"]/following-sibling::span[@class="custom-control-description"]').click()
    scroll('//div[@id="mthResultsStealPages"]')
    boton_estrato = driver.find_element(By.XPATH, '//div[@class="m2-select__value-container m2-select__value-container--is-multi css-1hwfws3"]').click()
    scroll('//div[@id="react-select-9-option-7"]')
    boton_filtro_estrato_4 = driver.find_element(By.XPATH, '//div[@id="react-select-9-option-3"]').click()
    boton_filtro_estrato_5 = driver.find_element(By.XPATH, '//div[@id="react-select-9-option-4"]').click()    
    #ACA HAY UN PROBLEMA, PORQUE EMPIEZA A CARGAR EL FILTRO DE ESTRATO 4 Y 5 Y CIERRA EL DROP DOWN, ENTONCES YA NO DEJA CLICKEAR EL 6
    boton_filtro_estrato_6 = driver.find_element(By.XPATH, '//div[@id="react-select-9-option-5"]').click()

In [5]:
#Función para scrapear página principal y obtener urls de los apartamentos.
def home_page_scrap():
    apartments_web_elements = driver.find_elements(By.XPATH, '//li[@class="sc-gPEVay dibcyk"]/div[@class="card-header"]/a')
    apartments_urls = []
    for element in apartments_web_elements:
        href = element.get_attribute('href')
        apartments_urls.append(href)
    return apartments_urls

In [6]:
#Función para scrapear página de cada apartamento.
def apartment_page_scrap(apartment_url): 
    driver.execute_script("window.open('');")
    driver.switch_to.window(driver.window_handles[1])
    driver.get(apartment_url)
#Reestablecer valores a nan 
    habitaciones = np.nan
    banos = np.nan
    estrato = np.nan
    codigo_inmueble = np.nan
    barrio = np.nan
    antiguedad = np.nan
    area_construida = np.nan
    area_privada = np.nan
    valor_arriendo = np.nan
    valor_administracion = np.nan
    parqueadero = np.nan
    sector = np.nan

#Obtener datos resaltados (habitaciones, banos y estrato).
    datos_resaltados_we = driver.find_elements(By.XPATH, '//li[@class="Li-sc-1knn373-0 ctJpwh"]/div[contains(@class, "py-2")]//h2')[1:]
    datos_limpios = []
    
    for i in datos_resaltados_we:
        name = i.text.split('\n')[1]
        value = i.text.split('\n')[0]
        datos_limpios.append({'name': name, 'value': value })

    
#Obtener datos principales(codigo_inmueble, barrio, ...sector).
    datos_principales_we = driver.find_elements(By.XPATH, '//div[@class="Col-sc-14ninbu-0 lfGZKA mb-3 pb-1 col-12 col-lg-3"]')  
    
    for i in datos_principales_we:
        name = i.find_element(By.XPATH, './h3').text
        value = i.find_element(By.XPATH, './p').text
        datos_limpios.append({'name': name, 'value': value })   

#Evaluar que estén todos los datos y que coincidan.

    for i in datos_limpios:
        if 'Habitaciones' in i['name']:
            habitaciones = i['value']
        if 'Baños' in i['name']:
            banos = i['value']
        if 'Estrato' in i['name']:
            estrato = i['value']
        if 'Código inmueble' in i['name']:
            codigo_inmueble = i['value']
        if 'Barrio común' in i['name']:
            barrio = i['value'].title()
        if 'Antigüedad' in i['name']:
            antiguedad = i['value']
        if 'Área construida' in i['name']:
            area_construida = i['value'].split(' m')[0]
        if 'Área privada' in i['name']:
            area_privada = i['value'].split(' m')[0]
        if 'Valor arriendo' in i['name']:
            valor_arriendo = i['value'].split('$')[1].replace('.','')
        if 'Valor administración' in i['name']:
            valor_administracion = i['value'].split('$')[1].replace('.','')
        if 'Parqueaderos' in i['name']:
            parqueadero = i['value']
    
    
    sector = driver.find_element(By.XPATH, '//p[@class="P-sc-31hrrf-0 hGwghD card-subtitle"]').text
    if 'Sector' in sector:
        sector = sector.split(',')[0].split('Sector ')[1]
    else:
        sector = sector.split('Área')[0].strip()
        
#Anexar información extraida de cada apartamento al consolidado. 
    info.append({'codigo_inmueble' : codigo_inmueble,
         'sector' : sector,
         'barrio' : barrio,
         'estrato': estrato, 
         'valor_arriendo' : valor_arriendo,
         'valor_administracion' : valor_administracion,
         'area_construida' : area_construida,
         'area_privada' : area_privada,
         'antiguedad' : antiguedad,
         'habitaciones': habitaciones,
         'banos': banos,       
         'parqueadero' : parqueadero,
         'url': apartment_url
        })
    driver.close()
    driver.switch_to.window(driver.window_handles[0])

In [7]:
#Función para scrapear todas las páginas de apartamentos según el filtro elegido.
def obtener_info():
    while True:
        delay = 10
        WebDriverWait(driver, delay).until(EC.presence_of_element_located((By.XPATH, '//li[@class="sc-gPEVay dibcyk"]/div[@class="card-header"]/a')))
        apartments_urls = home_page_scrap()
        for apartment in apartments_urls:
              apartment_page_scrap(apartment)
        #Click en el botón para continuar con la siguiente página:
        scroll('//h2[@class="H2-kplljn-0 igCxTv title-prefooter upper-text "]')
        next = driver.find_element(By.XPATH, '//li[@class="item-icon-next page-item"]/a[@class="sc-bdVaJa ebNrSm page-link"][@aria-label="Next"]').click()

In [ ]:
#Ejecutar código
info = []
url = 'https://www.metrocuadrado.com/apartamentos/arriendo/bogota/'
delay = 15
driver_path = '../chromedriver_mac64/chromedriver'
chrome_options = Options()
chrome_options.add_argument("--window-size=1920x1080")
driver = webdriver.Chrome(service = Service(driver_path))
driver.get(url)
WebDriverWait(driver, delay).until(EC.presence_of_element_located((By.XPATH, '//li[@class="sc-gPEVay dibcyk"]/div[@class="card-header"]/a')))
page_filter()
obtener_info()
driver.close()

In [30]:
#Validar como quedó el diccionario
info[:10]

[{'codigo_inmueble': '359-M4437421',
  'sector': 'Chapinero Alto',
  'barrio': 'Marly Chapinero',
  'estrato': '4',
  'valor_arriendo': '7623530',
  'valor_administracion': '1928000',
  'area_construida': '184',
  'area_privada': '184',
  'antiguedad': 'Entre 10 y 20 años',
  'habitaciones': '1',
  'banos': '2',
  'parqueadero': nan,
  'url': 'https://www.metrocuadrado.com/inmueble/arriendo-apartamento-bogota-marly-1-habitaciones-2-banos/359-M4437421'},
 {'codigo_inmueble': '2941-M3670392',
  'sector': 'Normandía',
  'barrio': 'Normandia',
  'estrato': '4',
  'valor_arriendo': '1300000',
  'valor_administracion': '280000',
  'area_construida': '56',
  'area_privada': '54',
  'antiguedad': 'Más de 20 años',
  'habitaciones': '2',
  'banos': '2',
  'parqueadero': '1',
  'url': 'https://www.metrocuadrado.com/inmueble/arriendo-apartamento-bogota-normandia-2-habitaciones-2-banos-1-garajes/2941-M3670392'},
 {'codigo_inmueble': '2214-M4489363',
  'sector': 'Cedritos',
  'barrio': 'Cerros',
  

In [26]:
#Contar de cuántos apartamentos se extrajo la información.
len(info)

3694

In [27]:
#Convertir lista de diccionarios a un dataframe con pandas.
df = pd.DataFrame(info)

In [28]:
#Revisar que todo esté bien visualizando los últimos registros
df.tail()

,codigo_inmueble,sector,barrio,estrato,valor_arriendo,valor_administracion,area_construida,area_privada,antiguedad,habitaciones,banos,parqueadero,url
3689,175-M4428546,Chicó,Chico Norte,5,4000000,604000,63,63,Entre 10 y 20 años,2,2,NaN,https://www.metrocuadrado.com/inmueble/arriend...
3690,MC4260493,Cedritos,Belmira,4,1600000,301400,43,43,Más de 20 años,1,1,1,https://www.metrocuadrado.com/inmueble/arriend...
3691,MC4454730,Chicó,Santa Bárbara Occidental,6,6200000,NaN,107,88,Entre 0 y 5 años,2,2,2,https://www.metrocuadrado.com/inmueble/arriend...
3692,MC4491486,Salitre Modelia,Modelia,4,1500000,NaN,65,60,Entre 10 y 20 años,2,2,NaN,https://www.metrocuadrado.com/inmueble/arriend...
3693,MC4491859,Santa Bárbara,Chico San Patricio Chicó,6,3000000,NaN,90,90,Más de 20 años,2,3,1,https://www.metrocuadrado.com/inmueble/arriend...


In [29]:
#Exportar dataframe como un archivo csv.
file_path = '/Users/diegotorres/Documents/Learning/Proyectos practicos/Bogota apartments/Propio/data_clean.csv'
df.to_csv(file_path, encoding='utf-8', index=False)